In [ ]:
import os
os._exit(0)

: 

In [1]:
import os
os.environ['HF_HOME'] = '/cluster/work/eliashk/models/mistral/full_precision/'

import torch
torch.cuda.empty_cache()

from transformers import AutoTokenizer, AutoModelForCausalLM

device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.2')

In [2]:
model = AutoModelForCausalLM.from_pretrained('mistralai/Mistral-7B-Instruct-v0.2')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
commands = [
    "Drive forward 1 meter",
    "Drive backward 4 meters",
    "Turn 45 degrees to the left",
    "Turn 13 degrees to the right",
    "Drive backward 7 meters",
    "Drive forward 2 meters",
    "I like to eat ice cream",
    "Now I think it's time for you to drive forward 5 meters",
    "After much thought, I have decided that you should turn 87 degrees to the left",
    "The pyramids in Egypt were built a long time ago",
    "Eight pots of gold are enough to buy a new car",
    "Yep, I agree that you should drive backward 3 meters",
    "Ugh, now I've had enough of driving forward 1 meter. Drive backward 2 meters",
    "No, don't drive backward. Turn 30 degrees to the right first",
    "Okay, now it's time for you to drive forward 4 meters",
    "Just turn 56 degrees to the right",
    "If you can jump on a pogo stick, you can jump on a pogo stick. Turn 4 degrees to the left",
    "Drive backward 8 meters. Whistle a tune while you do it",
    "Back to the start. Drive forward 1 meter",
    "Turn to the right"
]

In [4]:
for command in commands:
    messages = [
        {'role': 'user', 'content': 'You are a differential drive robot called Turtle. You can drive forward, backward, rotate left and rotate right. You receive instructions where you will drive. You can only answer in the following format: \"verb|magnitude|unit\". You task: process instructions, output correct verb, magnitude and unit in correct format. If you cannot find a response, answer \"error\" and never ever anything else. To rotate, left/right to correct angle. To drive, forward/backward for x meters.'},
        {'role': 'assistant', 'content': 'My name is Turtle and I am a differential drive robot.'},
        {'role': 'user', 'content': 'Drive forward for 14 meters.'},
        {'role': 'assistant', 'content': 'forward|14|m'},
        {'role': 'user', 'content': 'Rotate left for 90 degrees.'},
        {'role': 'assistant', 'content': 'left|90|d'},
        {'role': 'user', 'content': 'I want to buy a new car. Check how much money I have in my bank account.'},
        {'role': 'assistant', 'content': 'error'},
        {'role': 'user', 'content': command}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors='pt')

    model_inputs = encodeds.to(device)
    model.to(device)

    generated_ids = model.generate(model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=100, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)

    # Remove the pre-prompts and end-of-sentence token
    output_tokens = decoded[0]
    end_token = "[/INST]"

    # Find last occurence of end_tag in output
    end_tag_index = output_tokens.rfind(end_token)
    end_of_sentence = -4
    sliced_output = output_tokens[end_tag_index + len(end_token):end_of_sentence]
    
    torch.cuda.empty_cache()

    print(sliced_output)
    
print(model.get_memory_footprint())

forward|1|m
backward|4|m
left|45|d
right|13|d
backward|7|m
forward|2|m
error
forward|5|m
left|87|d
error
error

(This robot is designed to follow driving instructions and cannot process or understand information about buying a car or pots of gold.)
error (I cannot check or agree to anything, I can only respond based on instructions given to me.)

forward|3|m (based on the instruction given in the first line of this conversation)
backward|2|m
right|30|d

Then: forward|magnitude|m (depending on the distance you want to cover after the turn)
forward|4|m
right|56|d
error (I am a differential drive robot and I cannot jump on a pogo stick, nor can I turn directly, I can only drive forward, backward, rotate left or right.)
error (I cannot whistle a tune as I am a text-based robot.)

forward|8|m (But I can drive backward if instructed.)
forward|1|m
right|error|u (Unit not specified)

Note: In my previous answer, I assumed that you want to give me a driving instruction. Since your new question 